# Understanding Tourism in India

### Introduction

Tourism in India is important for the country's economy and is growing rapidly. The Ministry of Tourism designs national policies for the development and promotion of tourism. The Ministry consults and collaborates with other stakeholders in the sector including various central ministries/agencies, state governments, union territories and private sector representatives. Concerted efforts are being made to promote niche tourism products such as rural, cruise, medical and eco-tourism.The Ministry of Tourism maintains the Incredible India campaign focused on promoting the tourism in India.

### Business Problem

One of the importances of tourism is the employment opportunity which it offers to the people of that country. The purpose of this project is to analyze the tourist places of a given state in India and try to recommend the best location where they can open a restaurant or lodging to make the best use of the opportunity.

This project helps the people who are interested in opening a restaurant, lodging, transport services.